In [48]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *


import numpy as np
from langdetect import detect
import translitcodec
import codecs

# 1. Standardowa analiza

In [52]:
path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
# df3 = df3.loc[df3["startYear"].astype(int).between(1800, 2024), ["tconst", "titleType", "startYear"]]

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [54]:
df3.loc[df3["titleType"] == 'movie']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,<NA>,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,<NA>,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,<NA>,100,<NA>
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,<NA>,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,<NA>,90,Drama
...,...,...,...,...,...,...,...,...,...
10864942,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,<NA>,57,Documentary
10864969,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,<NA>,100,Documentary
10864981,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,<NA>,<NA>,Comedy
10864991,tt9916730,movie,6 Gunn,6 Gunn,0,2017,<NA>,116,Drama


In [49]:
path_to_file = "data/title.akas.tsv.gz"
df1 = load_to_dataframe(path_to_file)
df1 = df1.drop(columns=["language", "types", "attributes"])

path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file).dropna()
df3 = df3.loc[df3["startYear"].astype(int).between(1800, 2024), ["tconst", "titleType", "startYear"]]

path_to_file = "data/title.ratings.tsv.gz"
df7 = load_to_dataframe(path_to_file)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [30]:
df = df1.merge(df3,
              how="left",
              left_on="titleId",
              right_on="tconst"
)

# wybieramy tylko "movie" i niektóre kolumny
df = df.loc[df["titleType"] == "movie", ["titleId", "title", "region", "startYear", "isOriginalTitle"]]

In [19]:
# zamieniamy wszystkie tytuły na lowercase i dekodujemy narodowe znaki
df["title"] = df["title"].astype(str).str.lower().apply(lambda x: codecs.encode(str(x), 'translit/short'))

# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[df["isOriginalTitle"] == 0].merge(
    df.loc[df["isOriginalTitle"] == 1],
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "region_y", "startYear_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
disambig = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count()["region"] == 1

df_with_known_countries = df_think.loc[df_think["titleId"].isin(disambig.index[disambig == True])]

df_analysis = df_with_known_countries.merge(df7,
                                            how="left",
                                            left_on="titleId",
                                            right_on="tconst"
).drop(
    columns=["tconst"]
).dropna()

In [20]:
df_analysis

,titleId,title,region,startYear,averageRating,numVotes
0,tt0000147,the corbett-fitzsimmons fight,US,1897,5.2,517.0
1,tt0000502,bohemios,ES,1905,4.4,17.0
2,tt0000591,l'enfant prodigue,FR,1907,5.7,25.0
3,tt0000615,robbery under arms,AU,1907,4.3,25.0
4,tt0000630,amleto,IT,1908,2.9,27.0
...,...,...,...,...,...,...
460232,tt9916160,drommeland,NL,2019,6.2,52.0
460234,tt9916170,o ensaio,BR,2019,7.0,7.0
460237,tt9916270,il talento del calabrone,IT,2020,5.8,1488.0
460238,tt9916538,kuambil lagi hatiku,ID,2019,8.3,9.0


# 2. Model językowy

In [6]:
# wspieramy rozpoznawanie języka przez bibliotekę 'langdetect'
def mydetect(x):
    try:
        return detect(x).upper()
    except:
        return pd.NA

# funkcje do .apply()
def myfunc_EC(x):
    if x[5] != 'ES':
        x[2] = x[5]
    return x

def myfunc_TR(x):
    if x[5] != 'TR':
        x[2] = x[5]
    return x
    
# usuwamy dziwne filmy z Ekwadoru
df_EC = df_analysis.loc[df_analysis["region"] == 'EC'].copy()
df_EC.loc[:, "detected"] = df_EC["title"].apply(mydetect)

false_EC = df_EC.loc[df_EC["detected"] != 'ES']["titleId"]
df_EC = df_EC.apply(myfunc_EC, axis=1).drop(columns=["detected"])

df_analysis.loc[df_EC.index] = df_EC

# 3. Zasadnicza część zadania

In [67]:
from importlib import reload
import task1
reload(task1)
from task1 import *

In [68]:
df, df3 = load_dataframes(path1="data/title.akas.tsv.gz",
                            path2="data/title.basics.tsv.gz",
                            path3="data/title.ratings.tsv.gz")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [66]:
df.head(10)

,titleId,title,region,isOriginalTitle
72,tt0000009,Miss Jerry,<NA>,1
73,tt0000009,Miss Jerry,AU,0
74,tt0000009,Miss Jerry,HU,0
75,tt0000009,Miss Jerry,US,0
76,tt0000009,Fräulein Jerry,DE,0
749,tt0000147,The Corbett-Fitzsimmons Fight,<NA>,1
750,tt0000147,The Corbett-Fitzsimmons Fight,US,0
751,tt0000147,Бой Корбетта и Фитцсиммонса,RU,0
2045,tt0000502,Bohemios,<NA>,1
2046,tt0000502,Bohemios,ES,0


In [42]:
df_analysis = prepareDataset(df, df3)

In [43]:
rank = answerRanks(df_analysis, Ks = [10, 100, 1000], Ns = range(10, 201, 10), file = "answer.txt")

In [44]:
rank

{10: {10: [],
  20: [],
  30: [],
  40: [],
  50: [],
  60: [],
  70: [],
  80: [],
  90: [],
  100: [],
  110: [],
  120: [],
  130: [],
  140: [],
  150: [],
  160: [],
  170: [],
  180: [],
  190: [],
  200: [],
  0: []},
 100: {10: [],
  20: [],
  30: [],
  40: [],
  50: [],
  60: [],
  70: [],
  80: [],
  90: [],
  100: [],
  110: [],
  120: [],
  130: [],
  140: [],
  150: [],
  160: [],
  170: [],
  180: [],
  190: [],
  200: [],
  0: []},
 1000: {10: [],
  20: [],
  30: [],
  40: [],
  50: [],
  60: [],
  70: [],
  80: [],
  90: [],
  100: [],
  110: [],
  120: [],
  130: [],
  140: [],
  150: [],
  160: [],
  170: [],
  180: [],
  190: [],
  200: [],
  0: []}}

In [73]:
rank = {}
for K in [10, 100, 1000]:
    print(f"Threshold: {K}")
    rank[K] = {}
    for N in range(10, 201, 10):
        pa = properAnalysis(df_analysis, K = K, N = N)
        rank[K][N] = pa[:min(10, len(pa))]
        print()
    print("-"*40)

Threshold: 10
Top 10 countries - ranking
1. United States
2. Spain
3. India
4. Norway
5. El Salvador
6. Thailand

Top 20 countries - ranking
1. United States
2. India
3. Spain
4. Brazil
5. Italy
6. Norway
7. El Salvador
8. Thailand

Top 30 countries - ranking
1. United States
2. India
3. Brazil
4. Spain
5. Thailand
6. United Kingdom
7. Iran, Islamic Republic of
8. Italy
9. Norway
10. El Salvador

Top 40 countries - ranking
1. United States
2. India
3. Thailand
4. Brazil
5. Spain
6. United Kingdom
7. Canada
8. Colombia
9. Iran, Islamic Republic of
10. Italy

Top 50 countries - ranking
1. United States
2. India
3. Thailand
4. Brazil
5. Spain
6. United Kingdom
7. Italy
8. Canada
9. Colombia
10. Germany

Top 60 countries - ranking
1. United States
2. India
3. Spain
4. Italy
5. Thailand
6. Brazil
7. United Kingdom
8. Canada
9. Colombia
10. Germany

Top 70 countries - ranking
1. United States
2. India
3. Spain
4. Italy
5. Thailand
6. Brazil
7. Canada
8. United Kingdom
9. Iran, Islamic Republ